# *# Part 1 - Find lowest local price*

**1. Get name, address, open/close times, menu items, & prices for Village restaurant: https://www.yelp.com/biz/village-the-soul-of-india-hicksville**

In [1]:
#Importing Required Libraries
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json

In [2]:
#Feching full name of the restaurant
def get_name(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    try:
        return soup.find('h1').get_text()
    except:
        return None
name=get_name('https://www.yelp.com/biz/village-the-soul-of-india-hicksville')
print(name)

Village the soul of india


In [3]:
#Feching Address of Village restaurant

place_name = name
place_id="ChIJPYSDLXWBwokRHLcHIl02Kh8"
google_api_key = 'your_own_google_cloud_api_key'# Replace with your actual API keys
def get_place_details(api_key, place_id):                 # Function to get place details from Google Places API
    url = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&key={api_key}"
    response = requests.get(url)
    place_details = response.json()
    return place_details
place_details = get_place_details(google_api_key, place_id)# Get place details
address = place_details['result']['formatted_address']
print(f"Address: {address}")

Address: 11 W Marie St, Hicksville, NY 11801, USA


In [4]:
#Framing open/close times of Village restaurant
times = [
    ("11:30 AM", "5:00 PM"),
    ("Closed", "Closed"),
    ("11:30 AM", "5:00 PM"),
    ("11:30 AM", "5:00 PM"),
    ("11:30 AM", "5:00 PM"),
    ("11:30 AM", "5:00 PM"),
    ("11:30 AM", "5:00 PM")
]
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
# Create a DataFrame from the list of tuples
open_close_times_df = pd.DataFrame(times,index=days, columns=["Open_time", "Close_time"])

print(open_close_times_df)

          Open_time Close_time
Monday     11:30 AM    5:00 PM
Tuesday      Closed     Closed
Wednesday  11:30 AM    5:00 PM
Thursday   11:30 AM    5:00 PM
Friday     11:30 AM    5:00 PM
Saturday   11:30 AM    5:00 PM
Sunday     11:30 AM    5:00 PM


In [5]:
#Feching menu items and prices of Village restaurant
def scrape_menu(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    menu_items = soup.find_all('div', class_='menu-item')
    menu_data = []
    for item in menu_items:
        item_name = item.find('h4').text.strip()
        item_price = item.find('li', class_='menu-item-price-amount').text.strip()
        menu_data.append((item_name, item_price))
    return menu_data
url = 'https://www.yelp.com/menu/village-the-soul-of-india-hicksville'
menu_data = scrape_menu(url)
#converting menu data into DataFrame By using pandas
df = pd.DataFrame(menu_data, columns=['item_name', 'item_price'])
df['item_name'] = df['item_name'].str.lower()
df['item_price'] = df['item_price'].str.replace('$', '').astype(float)

df.head()

,item_name,item_price
0,medu vada,6.00
1,medu vada in sambar bowl,6.99
2,medu vada in rasam bowl,6.99
3,combo,5.99
4,combo in sambar bowl,6.99


...........................................................................................

.........


**2. Find top-rated 5 restaurants within 2 km serving similar items on their menu (use Yelp or Google Maps.)**

In [6]:
def find_nearby_restaurants(api_key, location, radius=2000):
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        'location': location,
        'radius': radius,
        'type': 'restaurant',
        'key': api_key
    }
    response = requests.get(url, params=params)
    results = response.json().get('results', [])
    restaurants = []
    for place in results:
        name = place.get('name')
        address = place.get('vicinity')
        rating = place.get('rating')
        restaurants.append((name, address, rating))
    return restaurants

api_key = 'your_own_google_cloud_api_key'   # Replace with your actual API key

location = '40.76655,-73.52353'                       # Coordinates for Village - The Soul of India
restaurants = find_nearby_restaurants(api_key, location)
print("--------------------------All Restarent near village---------------------------------- ")
for restaurant in restaurants:
    print(f"Name: {restaurant[0]}, Address: {restaurant[1]}, Rating: {restaurant[2]}")
print("---------------------------------------------------------------------------------------")
print("Top 5 restaurents")
top_restaurants = sorted(restaurants, key=lambda x: x[2], reverse=True)[:5]
nearest_df = pd.DataFrame(top_restaurants, columns=['Name', 'Address', 'Rating'])
nearest_df.head()

--------------------------All Restarent near village---------------------------------- 
Name: Buffalo Wild Wings, Address: 358 North Broadway, Hicksville, Rating: 3.8
Name: Chuck E. Cheese, Address: 11-15 Hanover Place Delco Plaza, Hicksville, Rating: 4.2
Name: Chili & Curry, Address: 106 Woodbury Road, Hicksville, Rating: 4.1
Name: House of Dosas, Address: 416 South Broadway, Hicksville, Rating: 4
Name: Panera Bread, Address: 201 Broadway Mall, Hicksville, Rating: 4.2
Name: Subway, Address: 495 South Broadway Suite 4, Hicksville, Rating: 2.6
Name: Subway, Address: 358B Broadway Food Court Store No 200, Hicksville, Rating: 4
Name: Smashburger, Address: 180 West Old Country Road, Hicksville, Rating: 4.3
Name: Broadway Diner, Address: 287 North Broadway, Hicksville, Rating: 4
Name: Papa Johns Pizza, Address: 285 South Broadway 4 #4, Hicksville, Rating: 3.1
Name: Red Lobster, Address: 1 Nevada Street, Hicksville, Rating: 4.1
Name: Sakana, Address: 68 North Broadway, Hicksville, Rating: 4.

,Name,Address,Rating
0,Sakana,"68 North Broadway, Hicksville",4.5
1,Smashburger,"180 West Old Country Road, Hicksville",4.3
2,Chuck E. Cheese,"11-15 Hanover Place Delco Plaza, Hicksville",4.2
3,Panera Bread,"201 Broadway Mall, Hicksville",4.2
4,Sunshine Hicksville,"275 West Old Country Road, Hicksville",4.2


# 3. Display menu items & prices for Village + each restaurant


In [7]:
#Sakana Chinese & Japanese Cuisine
url="https://order.newsakana.com/order/japanese-cuisine/japanese-appetizers"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
menu_items = soup.find_all('a', class_='eds_list-item')

menu_data = []
for item in menu_items:
    item_name = item.contents[0].strip()
    item_price = item.find('span', class_='ms-auto').text.strip()
    menu_data.append((item_name, item_price))
df1 = pd.DataFrame(menu_data, columns=['item_name', 'item_price'])
df1['item_name'] = df1['item_name'].str.lower()
print(df1)

                        item_name item_price
0             japanese appetizers         43
1                           salad          8
2            rolls and hand rolls         39
3    a la carte sushi and sashimi         16
4       sushi and sashimi entrees         18
..                            ...        ...
100              large party tray          5
101                lunch specials         32
102                    side order          8
103                        drinks          6
104                      desserts          5

[105 rows x 2 columns]


In [8]:
#Smashburger
def scrape_menu(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all menu items
    menu_items = soup.find_all('div', class_='menu-item')

    menu_data = []
    for item in menu_items:
        item_name = item.find('h4').text.strip()
        item_price = item.find('li', class_='menu-item-price-amount').text.strip()
        menu_data.append((item_name, item_price))

    return menu_data

# Replace 'https://example.com/menu' with the actual URL
url = 'https://www.yelp.com/menu/smashburger-hicksville-3'
menu_data = scrape_menu(url)
df2 = pd.DataFrame(menu_data, columns=['item_name', 'item_price'])
df2['item_name'] = df2['item_name'].str.lower()
print(df2)

                                item_name item_price
0                    double classic smash     $12.29
1                           classic smash      $8.84
2               double all-american smash     $12.04
3                      all-american smash      $8.60
4                double bacon stack smash     $14.50
5                       bacon stack smash     $11.67
6              double spicy jalapeo smash     $14.50
7                     spicy jalapeo smash     $11.67
8                            veggie smash     $11.30
9                  double bbq bacon smash     $13.76
10                        bbq bacon smash     $10.69
11          double truffle mushroom smash     $14.50
12                 truffle mushroom smash     $11.67
13                crispy chicken sandwich     $10.69
14  scorchin' hot crispy chicken sandwich     $11.06
15                   chicken smash burger     $10.20
16            double chicken smash burger     $13.64
17                              3 tenders     

In [9]:
#Chuck E. Cheese
url='https://order.chuckecheese.com/store/8868?filter=100'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
menu_items = soup.find_all('div', class_='c-dhzjXW c-dhzjXW-jroWjL-align-center c-dhzjXW-awKDG-position-leading c-dhzjXW-iTKOFX-direction-vertical c-dhzjXW-ikdofoX-css c-kIzoUV c-kIzoUV-iJmjVY-css')
print(menu_items)
menu_data = []
for item in menu_items:
        item_name = item.find('h3', class_='c-fNwGMe').text.strip()
        item_price = item.find('strong').text.strip()
        menu_data.append((item_name, item_price))

df3 = pd.DataFrame(menu_data, columns=['item_name', 'item_price'])
df3['item_name'] = df3['item_name'].str.lower()
print(df3)

[<div class="c-dhzjXW c-dhzjXW-jroWjL-align-center c-dhzjXW-awKDG-position-leading c-dhzjXW-iTKOFX-direction-vertical c-dhzjXW-ikdofoX-css c-kIzoUV c-kIzoUV-iJmjVY-css"><div class="c-hovWgW"><img alt="Pepperoni" class="c-knEtSs" src="https://media.koala.io/resize?source=https://koala-api-production.s3.amazonaws.com/global-products/1x1_2feebbad-255e-4f02-9d48-15448bb90c24.jpg&amp;width=300&amp;quality=85&amp;format=auto"/></div><div class="c-dhzjXW c-dhzjXW-jroWjL-align-center c-dhzjXW-awKDG-position-leading c-dhzjXW-iTKOFX-direction-vertical c-dhzjXW-ikdofoX-css c-csVTcs c-csVTcs-ibsZnki-css"><h3 class="c-fNwGMe">Pepperoni</h3><div class="c-dhzjXW c-dhzjXW-jroWjL-align-center c-dhzjXW-bICGYT-position-center c-dhzjXW-ejCoEP-direction-horizontal c-dhzjXW-ihDzjWR-css"><strong>$24.58</strong><span>190 Cals</span></div></div></div>, <div class="c-dhzjXW c-dhzjXW-jroWjL-align-center c-dhzjXW-awKDG-position-leading c-dhzjXW-iTKOFX-direction-vertical c-dhzjXW-ikdofoX-css c-kIzoUV c-kIzoUV-iJmj

In [11]:
def scrape_menu(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all menu items
    menu_items = soup.find_all('div', class_='menu-item')

    menu_data = []
    for item in menu_items:
        item_name = item.find('h4').text.strip()
        item_price = item.find('div', class_='menu-item-prices').text.strip()
        menu_data.append((item_name, item_price))

    return menu_data

# Replace with the actual URL
url = 'https://www.yelp.com/menu/house-of-dosas-hicksville'
menu_data = scrape_menu(url)
df4 = pd.DataFrame(menu_data, columns=['item_name', 'item_price'])
df4['item_name'] = df4['item_name'].str.lower()
# Remove everything except numbers and decimal point
df4['item_price'] = df4['item_price'].str.replace(r"[^\d.]", "", regex=True)
df4['item_price'] = df4['item_price'].str.replace(r"1.252.25", "1.5", regex=True)
df4['item_price'] = df4['item_price'].astype(float)
print(df4)

                         item_name  item_price
0                             idly        3.50
1                        podi idly        4.50
2    mini idly w/ sambar or butter        4.50
3          idly in sambar or rasam        4.50
4                        rava idly        3.95
..                             ...         ...
113                      ice cream        3.50
114                          kulfi        3.50
115                  lychee fruits        4.00
116                        payasam        3.50
117                      rasamalai        3.50

[118 rows x 2 columns]


In [12]:
df4.head(100)

,item_name,item_price
0,idly,3.50
1,podi idly,4.50
2,mini idly w/ sambar or butter,4.50
3,idly in sambar or rasam,4.50
4,rava idly,3.95
...,...,...
95,mix veg uthappam,8.95
96,gunpowder uthappam *,8.95
97,jalepeno tomato gunpowder uthappam *,8.95
98,onion chilly mushroom garlic uthappam *,8.95


In [13]:
#Diwan Indian Restaurant & Bar
url='https://www.yelp.com/menu/diwan-indian-restaurant-and-bar-hicksville'
def scrape_menu(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all menu items
    menu_items = soup.find_all('div', class_='menu-item')

    menu_data = []
    for item in menu_items:
        item_name = item.find('h4').text.strip()
        item_price = item.find('div', class_='menu-item-prices').text.strip()
        menu_data.append((item_name, item_price))

    return menu_data
menu_data = scrape_menu(url)
df5 = pd.DataFrame(menu_data, columns=['item_name', 'item_price'])
df5['item_name'] = df5['item_name'].str.lower()
df5['item_price'] = df5['item_price'].str.replace(r"[^\d.]", "", regex=True)
df5['item_price'] = df5['item_price'].astype(float)
print(df5)

                                             item_name  item_price
0                 chana masala over rice lunch special       13.95
1               chicken makhni over rice lunch special       14.95
2    chicken tikka (tandoori)- over rice lunch special       12.95
3        chicken tikka masala- over rice lunch special       14.95
4                  dal makhni- over rice lunch special       13.95
..                                                 ...         ...
120                                             merlot       26.00
121                                         pinot noir       26.00
122                                       pinot grigio       26.00
123                                         chardonnay       26.00
124                                    sauvignon blanc       26.00

[125 rows x 2 columns]


In [14]:
#Lemonleaf Grill
url="https://www.yelp.com/menu/lemonleaf-grill-hicksville"

def scrape_menu(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all menu items
    menu_items = soup.find_all('div', class_='menu-item')

    menu_data = []
    for item in menu_items:
        item_name = item.find('h4').text.strip()
        parts = item_name.split(". ", 1)
        item_name=parts[1].strip() if len(parts) > 1 else item_name

        item_price = item.find('div', class_='menu-item-prices').text.strip()
        menu_data.append((item_name, item_price))

    return menu_data
menu_data = scrape_menu(url)
df6 = pd.DataFrame(menu_data, columns=['item_name', 'item_price'])
df6['item_name'] = df6['item_name'].str.lower()
df6['item_price'] = df6['item_price'].str.replace(r"[^\d.]", "", regex=True)
df6['item_price'] = df6['item_price'].astype(float)
print(df6)

                        item_name  item_price
0                        pad thai       13.75
1                 kuawteo pad gai       13.75
2                      pad see ew       13.75
3                        lard nar       13.75
4    pineapple coconut fried rice       13.75
..                            ...         ...
122                        sprite        3.25
123                  jasmine rice        3.75
124                    brown rice        3.75
125                  coconut rice        3.75
126              thai sticky rice        3.75

[127 rows x 2 columns]


In [15]:
url="https://www.yelp.com/menu/sexy-crab-cajun-seafood-and-sushi-bar-levittown"
def scrape_menu(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all menu items
    menu_items = soup.find_all('div', class_='menu-item')

    menu_data = []
    for item in menu_items:
        item_name = item.find('h4').text.strip()
        parts = item_name.split(".", 1)
        item_name=parts[1].strip() if len(parts) > 1 else item_name

        item_price = item.find('div', class_='menu-item-prices').text.strip()
        menu_data.append((item_name, item_price))

    return menu_data
menu_data = scrape_menu(url)
df7 = pd.DataFrame(menu_data, columns=['item_name', 'item_price'])
df7['item_name'] = df7['item_name'].str.lower()
print(df7)

               item_name item_price
0           hush puppies      $5.50
1     fried cheese stick      $6.60
2            onion rings      $6.60
3         popcorn shrimp     $12.10
4         fried calamari     $12.10
..                   ...        ...
176      cranberry juice      $3.85
177             lemonade      $3.85
178  unsweetened ice tea      $3.30
179              ice tea      $3.30
180               coffee      $4.40

[181 rows x 2 columns]


In [16]:
url="https://www.yelp.com/menu/ihop-hicksville"
def scrape_menu(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all menu items
    menu_items = soup.find_all('div', class_='menu-item')

    menu_data = []
    for item in menu_items:
        item_name = item.find('h4').text.strip()
        parts = item_name.split(".", 1)
        item_name=parts[1].strip() if len(parts) > 1 else item_name

        item_price = item.find('div', class_='menu-item-prices').text.strip()
        menu_data.append((item_name, item_price))

    return menu_data
menu_data = scrape_menu(url)
df8 = pd.DataFrame(menu_data, columns=['item_name', 'item_price'])
df8['item_name'] = df8['item_name'].str.lower()
print(df8)

                                             item_name item_price
0                     fully loaded french toast combos     $17.99
1                         the great circle of cinnamon     $18.59
2                                     berry'd treasure     $18.59
3                                 secret cookie butter     $18.59
4    junior adventurer's cookie butter french toast...     $10.79
..                                                 ...        ...
170                              milk & chocolate milk      $2.39
171                                    fountain drinks      $4.55
172                                          splashers      $5.75
173                          breakfast beverage bundle     $11.99
174                       lunch/dinner beverage bundle     $15.59

[175 rows x 2 columns]


In [17]:
df_final=df

common_items = pd.merge(df, df4, on='item_name', suffixes=('_restaurant1', '_restaurant2'))

print("Common Menu Items:")
print(common_items[['item_name', 'item_price_restaurant1', 'item_price_restaurant2']])

Common Menu Items:
                 item_name  item_price_restaurant1  item_price_restaurant2
0                    combo                    5.99                    3.50
1                dahi vada                    8.95                    3.95
2                     idly                    7.45                    3.50
3                podi idly                    8.95                    4.50
4                bhel puri                    8.95                    3.95
5             chinese dosa                   14.95                    8.50
6                sada dosa                   11.95                    4.95
7                 set dosa                   11.95                    6.95
8              masala dosa                   12.95                    5.50
9         butter sada dosa                   11.95                    5.95
10      butter masala dosa                   13.75                    6.50
11       ghee roast masala                   13.75                    7.50
12    

In [18]:
#house-of-dosas
df1=df
df2=df4
# Merge to find the lowest price for common items
merged = pd.merge(df1, df2, on='item_name', how='inner', suffixes=('_df1', '_df2'))
merged['lowest_price'] = merged[['item_price_df1', 'item_price_df2']].min(axis=1)

# Update df1 with the lowest prices
df1 = df1.set_index('item_name')
for _, row in merged.iterrows():
    df1.loc[row['item_name'], 'item_price'] = row['lowest_price']
df1 = df1.reset_index()
print(df1)

                    item_name  item_price
0                   medu vada        6.00
1    medu vada in sambar bowl        6.99
2     medu vada in rasam bowl        6.99
3                       combo        3.50
4        combo in sambar bowl        6.99
..                        ...         ...
181            masala tea hot        2.95
182            herbal tea hot        2.95
183                     shake        6.00
184           fresh lime soda        7.45
185               masala soda        7.45

[186 rows x 2 columns]


In [19]:
#
df2=df5
# Merge to find the lowest price for common items
merged = pd.merge(df1, df2, on='item_name', how='inner', suffixes=('_df1', '_df2'))
merged['lowest_price'] = merged[['item_price_df1', 'item_price_df2']].min(axis=1)

# Update df1 with the lowest prices
df1 = df1.set_index('item_name')
for _, row in merged.iterrows():
    df1.loc[row['item_name'], 'item_price'] = row['lowest_price']
df1 = df1.reset_index()
df1.head(1000)

,item_name,item_price
0,medu vada,6.00
1,medu vada in sambar bowl,6.99
2,medu vada in rasam bowl,6.99
3,combo,3.50
4,combo in sambar bowl,6.99
...,...,...
181,masala tea hot,2.95
182,herbal tea hot,2.95
183,shake,6.00
184,fresh lime soda,7.45


In [20]:
#
df2=df6
# Merge to find the lowest price for common items
merged = pd.merge(df1, df2, on='item_name', how='inner', suffixes=('_df1', '_df2'))
merged['lowest_price'] = merged[['item_price_df1', 'item_price_df2']].min(axis=1)

# Update df1 with the lowest prices
df1 = df1.set_index('item_name')
for _, row in merged.iterrows():
    df1.loc[row['item_name'], 'item_price'] = row['lowest_price']
df1 = df1.reset_index()
df1.head(1000)


,item_name,item_price
0,medu vada,6.00
1,medu vada in sambar bowl,6.99
2,medu vada in rasam bowl,6.99
3,combo,3.50
4,combo in sambar bowl,6.99
...,...,...
181,masala tea hot,2.95
182,herbal tea hot,2.95
183,shake,6.00
184,fresh lime soda,7.45


In [21]:
coconut_rice_price = df1.loc[df1['item_name'] == 'coconut rice', 'item_price'].iloc[0]
print(f"The price of coconut rice is: {coconut_rice_price}")

The price of coconut rice is: 3.75


In [22]:
df1 = df1.rename(columns={'item_price': 'Final lower Price ofter comparing'})
df1.head()

,item_name,Final lower Price ofter comparing
0,medu vada,6.00
1,medu vada in sambar bowl,6.99
2,medu vada in rasam bowl,6.99
3,combo,3.50
4,combo in sambar bowl,6.99


In [23]:

df1.to_csv('restaurant_menu.csv', index=False)